In [1]:
import fitz
import pandas as pd

In [20]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)


In [13]:
def find_column_separators(page):
    text_boxes = page.get_text("dict")["blocks"]
    column_separators = []

    for box in text_boxes:
        x0, y0, x1, y1 = box["bbox"]
        x_center = (x0 + x1) / 2

        # Assuming columns are separated by a certain distance
        if x_center not in column_separators:
            column_separators.append(x_center)

    return sorted(column_separators)

In [14]:
def extract_two_column_text(page, column_separators):
    text_boxes = page.get_text("dict")["blocks"]
    columns = {x: [] for x in column_separators}

    for box in text_boxes:
        x0, y0, x1, y1 = box["bbox"]
        x_center = (x0 + x1) / 2

        closest_column = min(column_separators, key=lambda x: abs(x - x_center))
        if "lines" in box:
            lines = [line["spans"] for line in box["lines"]]
            text = ' '.join(''.join(span['text'] for span in line) for line in lines)
            columns[closest_column].append(text)

    sorted_columns = sorted(columns.items(), key=lambda x: column_separators.index(x[0]))
    column_texts = ['\n\n'.join(col[1]) for col in sorted_columns]

    return column_texts

In [15]:
def extract_parts(pdf_path):
    pdf_document = fitz.open(pdf_path)
    toc = pdf_document.get_toc()
    parts_list = []

    for toc_item in toc:
        title = toc_item[1]
        start_page = toc_item[0]

        if toc.index(toc_item) < len(toc) - 1:
            end_page = toc[toc.index(toc_item) + 1][0] - 1
        else:
            end_page = len(pdf_document) - 1

        part_text = ''
        for page_num in range(start_page - 1, end_page):
            page = pdf_document.load_page(page_num)
            column_separators = find_column_separators(page)
            if len(column_separators) > 1:
                column_texts = extract_two_column_text(page, column_separators)
                part_text += "\n\n".join(column_texts) if column_texts else ""  # Separating columns by newline
            else:
                part_text += page.get_text()

        part_dict = {
            'title': title,
            'pages': f'{start_page}-{end_page}',
            'text': part_text
        }

        parts_list.append(part_dict)

    pdf_document.close()
    return parts_list

In [21]:
# Replace 'your_pdf_file.pdf' with the path to your PDF file
pdf_path = 'a.pdf'

# Extract parts from the PDF
parts = extract_parts(pdf_path)

# Create a DataFrame from the parts list
parts_df = pd.DataFrame(parts)

# Display the DataFrame
parts_df

,title,pages,text
0,Abbildung 1: Verwendete Methoden der von Kaggl...,1-0,
1,Abbildung 2: Schematische Darstellung eines KN...,1-0,
2,Abbildung 3: Schematische Darstellung eines En...,1-0,
3,Abbildung 4: Entwicklung der Publikationen im ...,1-0,
4,Abbildung 5: Anteil der Deep Learning Publikat...,1-0,
5,Abbildung 6: Kompetenzlandkarte im Bereich ML ...,1-0,
6,Abbildung 7: Entwicklung der Patentfamilien zu...,1-0,
7,Abbildung 8: Anzahl der deutschen Unternehmen ...,1-0,
8,Abbildung 9: Entwicklung der Patentfamilien fü...,1-0,
9,Abbildung 10: Anwendungsbranchen für Maschinel...,1-0,


In [22]:
parts

[{'title': 'Abbildung 1: Verwendete Methoden der von Kaggle befragten Data Scientists und ML-Fachleute3',
  'pages': '1-0',
  'text': ''},
 {'title': 'Abbildung 2: Schematische Darstellung eines KNN, hier in Form eines Feed-forward Network, eigene Darstellung',
  'pages': '1-0',
  'text': ''},
 {'title': 'Abbildung 3: Schematische Darstellung eines Entscheidungsbaums6',
  'pages': '1-0',
  'text': ''},
 {'title': 'Abbildung 4: Entwicklung der Publikationen im Bereich ML für Deutschland (DE), Frankreich (FR), Großbritannien (UK), USA (US) und China (CN) sowie die restlichen EU-Staaten und übrige Länder',
  'pages': '1-0',
  'text': ''},
 {'title': 'Abbildung 5: Anteil der Deep Learning Publikationen an gesamten ML-Publikationen, 2006-2016',
  'pages': '1-0',
  'text': ''},
 {'title': 'Abbildung 6: Kompetenzlandkarte im Bereich ML auf Basis der Anzahl von ML-bezogenen Artikeln in wissenschaftlichen Fachzeitschriften',
  'pages': '1-0',
  'text': ''},
 {'title': 'Abbildung 7: Entwicklung 